In [1]:
# Подключение библиотек
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Задание шапки таблицы данных для обучения
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [3]:
file = open('lr1_data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [5]:
# Извлечение функций
for filename in os.listdir(f'C:\\Users\\user\\Desktop\\Training'):
    songname = f'C:\\Users\\user\\Desktop\\Training\\{filename}'
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse=librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'        
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open('lr1_data.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

In [6]:
data = pd.read_csv('lr1_data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,background_0001.wav,0.624490,0.000158,3034.544804,2307.485058,5953.745943,0.250282,-738.071716,93.217445,-33.399258,...,-6.226457,12.462728,-12.907367,12.445766,-9.406123,10.940209,-3.830426,3.890665,-1.316001,NaN
1,background_0002.wav,0.745098,0.000071,3186.990847,2310.177615,6053.988874,0.272419,-789.601990,75.197586,-31.433498,...,-5.241376,10.665572,-9.677876,11.937750,-8.288611,9.191567,-3.542699,4.432508,0.822094,NaN
2,background_0003.wav,0.682624,0.000073,3389.104803,2308.878995,6213.184290,0.304189,-778.028931,73.797752,-33.068264,...,-7.054873,9.454710,-12.917912,11.743085,-9.146166,10.361667,-3.550290,4.598960,0.036682,NaN
3,background_0004.wav,0.760644,0.000089,3199.924891,2311.821234,6079.331513,0.279971,-780.314270,65.745621,-22.033247,...,-4.947610,8.886817,-7.817257,9.879170,-6.393075,6.643980,-2.605803,2.302400,1.334502,NaN
4,background_0005.wav,0.772042,0.000051,3215.602229,2310.978107,6092.315856,0.278374,-797.359863,72.981621,-32.363747,...,-4.422258,10.689495,-9.336390,11.430396,-8.628820,8.254793,-4.460248,3.480893,-0.209078,NaN


In [7]:
filename_search = {"background": ["background_"],
                   "bags": ["bags_", "bg_", "t_bags_"],
                   "door": ["door_", "d_", "t_door_"],
                   "keyboard": ["keyboard_", "t_keyboard_", "k_"],
                   "knocking_door": ["knocking_door_", "tt_kd_", "t_knocking_door_"],
                   "ring": ["ring_", "t_ring_"],
                   "speech": ["speech_"],
                   "tool": ["tool_"]}

In [8]:
# Получение ярлыков
def get_label_from_filename(filename):
    for key, value in filename_search.items():
        for val in value:
            if (filename.find(val) == 0):
                return key

directory = "C:\\Users\\user\\Desktop\\Training"
files = ([f.path for f in os.scandir(directory) if f.is_file()])
df = pd.DataFrame(columns=["path", "filename","label"])

for path in tqdm(files[:]):
    filename = os.path.splitext(os.path.basename(path).strip())[0]
    label = get_label_from_filename(filename)
    row = pd.Series([path, filename, label], index = df.columns)
    df = df.append(row, ignore_index=True)

  0%|          | 0/3648 [00:00<?, ?it/s]

In [9]:
df.head()

,path,filename,label
0,C:\Users\user\Desktop\Training\background_0001...,background_0001,background
1,C:\Users\user\Desktop\Training\background_0002...,background_0002,background
2,C:\Users\user\Desktop\Training\background_0003...,background_0003,background
3,C:\Users\user\Desktop\Training\background_0004...,background_0004,background
4,C:\Users\user\Desktop\Training\background_0005...,background_0005,background


In [10]:
data['label'].value_counts()

Series([], Name: label, dtype: int64)

In [11]:
data['label']=df['label']

In [12]:
data['label'].value_counts()

tool             1659
ring              713
bags              309
speech            276
door              244
keyboard          233
knocking_door     138
background         76
Name: label, dtype: int64

In [13]:
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,background_0001.wav,0.624490,0.000158,3034.544804,2307.485058,5953.745943,0.250282,-738.071716,93.217445,-33.399258,...,-6.226457,12.462728,-12.907367,12.445766,-9.406123,10.940209,-3.830426,3.890665,-1.316001,background
1,background_0002.wav,0.745098,0.000071,3186.990847,2310.177615,6053.988874,0.272419,-789.601990,75.197586,-31.433498,...,-5.241376,10.665572,-9.677876,11.937750,-8.288611,9.191567,-3.542699,4.432508,0.822094,background
2,background_0003.wav,0.682624,0.000073,3389.104803,2308.878995,6213.184290,0.304189,-778.028931,73.797752,-33.068264,...,-7.054873,9.454710,-12.917912,11.743085,-9.146166,10.361667,-3.550290,4.598960,0.036682,background
3,background_0004.wav,0.760644,0.000089,3199.924891,2311.821234,6079.331513,0.279971,-780.314270,65.745621,-22.033247,...,-4.947610,8.886817,-7.817257,9.879170,-6.393075,6.643980,-2.605803,2.302400,1.334502,background
4,background_0005.wav,0.772042,0.000051,3215.602229,2310.978107,6092.315856,0.278374,-797.359863,72.981621,-32.363747,...,-4.422258,10.689495,-9.336390,11.430396,-8.628820,8.254793,-4.460248,3.480893,-0.209078,background


In [14]:
# Преобразование данных для обучения
scaler = StandardScaler()
X = scaler.fit_transform(data.drop(['filename', 'label'],axis=1).values)

In [15]:
X

array([[ 0.90503261, -0.94819926,  0.53226763, ...,  0.63640432,
         0.07001937,  0.43826544],
       [ 1.82503306, -0.94955348,  0.71722489, ...,  0.68123133,
         0.15589143,  0.83070476],
       [ 1.3484812 , -0.94951433,  0.96244242, ...,  0.6800486 ,
         0.18227087,  0.68654525],
       ...,
       [ 0.14468148, -0.8897629 , -1.17219616, ...,  0.62378556,
        -1.3317461 , -0.17756825],
       [-0.00623053, -0.86132836, -0.90685575, ...,  0.35638808,
        -1.22963515, -0.34344363],
       [-0.05534983, -0.87700982, -1.04255573, ...,  0.38418734,
        -1.3485414 , -0.29691326]])

In [16]:
encoder = LabelEncoder()
y = encoder.fit_transform(data['label'].to_numpy())

In [17]:
y

array([0, 0, 0, ..., 5, 5, 5])

In [18]:
labels = ["background",
          "bags",
          "door",
          "keyboard",
          "knocking_door",
          "ring",
          "speech",
          "tool"]
encoder = LabelEncoder()
encoder.fit(labels)
encoder.classes_

array(['background', 'bags', 'door', 'keyboard', 'knocking_door', 'ring',
       'speech', 'tool'], dtype='<U13')

In [19]:
y = encoder.transform(df['label'].to_numpy())
data = data.assign(label=y)

In [20]:
data

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,background_0001.wav,0.624490,0.000158,3034.544804,2307.485058,5953.745943,0.250282,-738.071716,93.217445,-33.399258,...,-6.226457,12.462728,-12.907367,12.445766,-9.406123,10.940209,-3.830426,3.890665,-1.316001,0
1,background_0002.wav,0.745098,0.000071,3186.990847,2310.177615,6053.988874,0.272419,-789.601990,75.197586,-31.433498,...,-5.241376,10.665572,-9.677876,11.937750,-8.288611,9.191567,-3.542699,4.432508,0.822094,0
2,background_0003.wav,0.682624,0.000073,3389.104803,2308.878995,6213.184290,0.304189,-778.028931,73.797752,-33.068264,...,-7.054873,9.454710,-12.917912,11.743085,-9.146166,10.361667,-3.550290,4.598960,0.036682,0
3,background_0004.wav,0.760644,0.000089,3199.924891,2311.821234,6079.331513,0.279971,-780.314270,65.745621,-22.033247,...,-4.947610,8.886817,-7.817257,9.879170,-6.393075,6.643980,-2.605803,2.302400,1.334502,0
4,background_0005.wav,0.772042,0.000051,3215.602229,2310.978107,6092.315856,0.278374,-797.359863,72.981621,-32.363747,...,-4.422258,10.689495,-9.336390,11.430396,-8.628820,8.254793,-4.460248,3.480893,-0.209078,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,t_ring_0054.wav,0.464231,0.002534,1461.148750,1479.271880,2903.183868,0.084173,-507.316071,147.122482,-36.053429,...,-8.269979,-0.059439,-7.614376,-3.553323,-9.592006,-9.557157,-8.238001,-5.612866,-0.127256,5
3644,t_ring_0055.wav,0.508417,0.003464,1634.654728,1551.548570,3351.539913,0.097449,-534.645386,130.290634,-39.378330,...,-11.864310,-2.677356,-9.077792,-3.718090,-4.497273,-3.267391,-4.333245,-4.862713,-5.115257,5
3645,t_ring_0056.wav,0.524811,0.003920,1629.686388,1556.096755,3352.957782,0.098368,-532.348572,128.190491,-34.637394,...,-12.291412,-3.134912,-8.717246,-3.374793,-3.780036,-3.223276,-3.911421,-4.954324,-4.671196,5
3646,t_ring_0057.wav,0.505027,0.005750,1848.386083,1584.794573,3739.240723,0.126050,-504.904694,120.971931,-47.436543,...,-12.715235,-2.275867,-9.582973,-4.108831,-4.730484,-2.512986,-5.627739,-4.310015,-5.574922,5


In [21]:
# Разделение данных на выборки для обучения
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
# Архитектура модели

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               13824     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 186,954
Trainable params: 186,954
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Точность модели перед тренировкой
score = model.evaluate(X_train, y_train, verbose=0)
accuracy = 100*score[1]
print(accuracy)

3.1528443098068237


In [24]:
history = model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=1)

Epoch 1/30
23/23 [==============================] - 0s 5ms/step - loss: 1.2355 - accuracy: 0.6323 - val_loss: 0.6342 - val_accuracy: 0.7918
Epoch 2/30
23/23 [==============================] - 0s 4ms/step - loss: 0.4417 - accuracy: 0.8585 - val_loss: 0.3743 - val_accuracy: 0.8767
Epoch 3/30
23/23 [==============================] - 0s 4ms/step - loss: 0.2839 - accuracy: 0.9119 - val_loss: 0.2720 - val_accuracy: 0.9137
Epoch 4/30
23/23 [==============================] - 0s 3ms/step - loss: 0.2122 - accuracy: 0.9383 - val_loss: 0.2377 - val_accuracy: 0.9370
Epoch 5/30
23/23 [==============================] - 0s 4ms/step - loss: 0.1614 - accuracy: 0.9513 - val_loss: 0.2045 - val_accuracy: 0.9425
Epoch 6/30
23/23 [==============================] - 0s 3ms/step - loss: 0.1340 - accuracy: 0.9630 - val_loss: 0.1955 - val_accuracy: 0.9384
Epoch 7/30
23/23 [==============================] - 0s 3ms/step - loss: 0.1178 - accuracy: 0.9640 - val_loss: 0.1728 - val_accuracy: 0.9466
Epoch 8/30
23/23 [==

In [25]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Точность обучения: {0:.2%}".format(score[1]))
score = model.evaluate(X_test, y_test, verbose=0)
print("Точность тестирования: {0:.2%}".format(score[1]))

Точность обучения: 100.00%
Точность тестирования: 96.03%


In [26]:
# Создание файла с результами обучения на исходных данных

In [27]:
# Предсказанные метки классов
predictions=model.predict(X) 
predict_classes=np.argmax(predictions,axis=1)
print(predict_classes)

[0 0 0 ... 5 5 5]


In [28]:
# Предсказанный скор
def predict_prob(number):
    return np.sum(number[0])

predict_score = np.array(list(map(predict_prob, predictions)))
print(predict_score) 

[9.9946076e-01 9.9714088e-01 9.9952173e-01 ... 4.6964728e-06 3.8268860e-07
 5.1804608e-07]


In [29]:
# Формирование данных в необходимом формате
data['predict_classes']=predict_classes
data['predict_score']=predict_score

result=data[['filename','label', 'predict_score', 'predict_classes']]

In [30]:
result.head()

,filename,label,predict_score,predict_classes
0,background_0001.wav,0,0.999461,0
1,background_0002.wav,0,0.997141,0
2,background_0003.wav,0,0.999522,0
3,background_0004.wav,0,0.994003,0
4,background_0005.wav,0,0.996848,0


In [31]:
result.to_csv('result_lr1.csv')

In [32]:
# Тестовые данные 

In [33]:
# Задание шапки таблицы данных для обучения
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
# header += ' label'
header = header.split()

In [34]:
file = open('lr1_data_test.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [35]:
# Извлечение функций
for filename in os.listdir(f'C:\\Users\\user\\Desktop\\Test'):
    songname = f'C:\\Users\\user\\Desktop\\Test\\{filename}'
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse=librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'        
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    file = open('lr1_data_test.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

In [48]:
data_test = pd.read_csv('lr1_data_test.csv').reset_index()
data_test.head()

,index,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,Track,01.wav,0.419928,0.000664,1756.587717,1939.971949,4251.197658,0.080391,-622.857971,151.583878,...,-0.668716,-12.078215,5.486744,-17.639717,4.683621,-17.664104,-0.157782,-12.648448,-1.873771,-3.377900
1,Track,02.wav,0.439897,0.000685,1728.150954,1915.787817,4149.637881,0.082556,-619.838989,152.332993,...,-1.928678,-12.163959,6.138718,-16.800549,5.882452,-15.967912,1.211754,-11.318262,-2.335084,-4.202593
2,Track,03.wav,0.432539,0.000648,1830.583806,1950.131896,4328.838433,0.087647,-623.890564,149.021637,...,-3.357036,-13.861594,5.121989,-17.536833,5.861159,-17.894180,0.216881,-11.926983,-2.046184,-3.539523
3,Track,04.wav,0.428965,0.000648,1812.176767,1958.451938,4345.872351,0.084173,-625.420898,149.846832,...,-2.609134,-13.649405,5.464570,-17.490993,5.583587,-17.706490,0.250060,-12.626258,-2.761898,-3.964132
4,Track,05.wav,0.405623,0.000677,1844.206809,1957.616452,4341.326841,0.086221,-621.193237,144.522537,...,-3.589686,-14.021877,3.816517,-18.206089,5.427678,-16.939087,-0.630145,-11.812579,-2.997674,-3.999764


In [49]:
# Преобразование данных для обучения
scaler = StandardScaler()
X_test = scaler.fit_transform(data_test.drop(['filename', 'index'],axis=1).values)

In [50]:
X_test

array([[-0.68037028, -0.4562905 , -0.57787378, ..., -1.63956558,
         0.07704934, -0.36108275],
       [-0.48851695, -0.4558195 , -0.61969425, ..., -1.29992016,
        -0.05125713, -0.69919198],
       [-0.55920762, -0.4566325 , -0.46905157, ..., -1.45534898,
         0.02909557, -0.4273452 ],
       ...,
       [ 1.4051682 , -0.2788046 ,  0.24469567, ...,  1.13121285,
         0.11303367,  1.19530965],
       [ 0.75733067, -0.38888366,  0.28850573, ...,  0.24658903,
        -0.17170428,  0.16066496],
       [ 1.21287019, -0.43809124,  0.5675949 , ...,  0.55422835,
         0.14269978, -0.01924005]])

In [51]:
predictions_test=model.predict(X_test) 
predict_classes=np.argmax(predictions_test,axis=1)
print(predict_classes)

[7 7 7 7 3 3 7 1 1 1 7 7 2 2 7 6 2 7 3 0 1 2 7 5 2 7 2 2 2 2 2 1 2 7 5 2 2
 5 2 1 2 2 0 7 2 7 7 1 2 2 2 7 7 0 2 7 7 7 3 3 3 3 3 3 0 1 1 3 3 1 3 3 5 7
 7 0 3 3 7 7 7 7 7 7 7 7 7 7 1 7 7 7 7 7 7 7 7 0 7 7 3 7 7 7 7 7 7 4 0 4 7
 2 4 4 3 4 4 4 4 7 2 7 4 4 7 7 4 4 4 4 0 4 4 7 7 4 7 7 7 4 2 7 4 7 7 7 7 7
 7 4 4 2 3 6 6 4 7 7 7 2 7 7 7 7 7 4 2 4 4 4 4 5 5 2 7 5 5 5 5 5 5 5 5 5 5
 3 3 5 5 5 5 5 5 5 5 5 7 5 5 5 5 5 5 5 5 5 5 7 5 5 5 5 5 5 5 5 5 0 7 5 5 5
 7 5 7 5 5 5 7 0 7 5 5 5 5 5 5 5 5 5 7 5 5 5 5 5 5 5 5 5 3 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 5 7 7 7 7 5 7 7 7 7 7 5 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 6 2 2 7 6 6 6 6
 2 6 6 6 2 6 7 6 5 6 5 7 7 7 5 6 6 1 6 6 6 7 7 7 6 7 5 7 7 7 5 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 7 7 7 7 7 1 3 7 1 1 1 1
 1 7 7 7 7 7 1 7 7 7 7 7 7 7 7 7 7 2 2 1 1 7 7 1 3]


In [52]:
def predict_prob(number):
    return np.sum(number[0])

predict_score = np.array(list(map(predict_prob, predictions_test)))
print(predict_score) 

[1.49565837e-09 1.07730962e-07 1.83800111e-07 2.20576037e-08
 4.04462526e-08 1.07110122e-06 2.20158006e-12 1.28623633e-05
 1.11558312e-03 7.78552785e-05 1.47514768e-07 9.77074933e-07
 9.86232553e-05 2.12296095e-06 2.23621246e-08 3.57313268e-03
 1.48960695e-04 4.43306546e-12 1.30348926e-04 9.59282637e-01
 5.91583703e-05 1.47592564e-05 5.98693077e-08 7.72910062e-06
 1.14824090e-07 2.71613374e-11 3.31832816e-05 2.34010804e-05
 7.99451527e-07 1.99173247e-10 4.77922448e-08 9.41810024e-04
 1.52648972e-05 1.37579127e-03 7.82727511e-06 4.97829615e-06
 3.33124859e-07 2.20830673e-10 4.13170601e-05 1.21657990e-06
 5.33807906e-04 6.77531091e-07 8.39473367e-01 5.64170257e-08
 8.88706472e-06 1.48700423e-07 4.15301457e-07 1.17422028e-08
 3.09581232e-08 5.68604586e-09 7.43681206e-09 4.60474181e-14
 1.68019006e-12 5.55969477e-01 3.97907970e-05 5.75681603e-12
 2.87069305e-12 1.43517589e-15 2.44658606e-14 4.64267691e-09
 3.05777945e-08 2.11513704e-08 2.68819687e-08 6.67400579e-09
 9.09266472e-01 2.416246

In [66]:
# Формирование данных в необходимом формате
data_test['predict_classes']=predict_classes
data_test['predict_score']=predict_score

result_test=data_test[['filename', 'predict_score', 'predict_classes']]

In [67]:
result_test

,filename,predict_score,predict_classes
0,01.wav,1.495658e-09,7
1,02.wav,1.077310e-07,7
2,03.wav,1.838001e-07,7
3,04.wav,2.205760e-08,7
4,05.wav,4.044625e-08,3
...,...,...,...
427,95.wav,7.251053e-16,1
428,96.wav,2.201824e-07,7
429,97.wav,1.065607e-08,7
430,98.wav,2.204313e-04,1


In [68]:
mapping = {0:'background', 1:'bags', 2:'door', 3:'keyboard', 4:'knocking_door', 5:'ring', 6:'speech', 7:'tool'}
result_test['predict_classes'] = result_test['predict_classes'].map(mapping)

In [69]:
result_test

,filename,predict_score,predict_classes
0,01.wav,1.495658e-09,tool
1,02.wav,1.077310e-07,tool
2,03.wav,1.838001e-07,tool
3,04.wav,2.205760e-08,tool
4,05.wav,4.044625e-08,keyboard
...,...,...,...
427,95.wav,7.251053e-16,bags
428,96.wav,2.201824e-07,tool
429,97.wav,1.065607e-08,tool
430,98.wav,2.204313e-04,bags


In [70]:
result_test.to_csv('result_lr1_test.csv')